In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor, ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVR, SVC
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor, StackingClassifier
import time
import psutil

# Đọc dữ liệu từ file
df_train = pd.read_csv('/kaggle/input/ail303m/train.csv')
df_test = pd.read_csv('/kaggle/input/ail303m/final1 - Copy.csv')

# Drop cột 'type'
df_train.drop('type', axis=1, inplace=True)
df_test.drop('type', axis=1, inplace=True)

In [3]:
# Standardize the data
scaler = StandardScaler()

# Specify the features and the target variable
features = [col for col in df_train.columns if col not in ['quality', 'id', 'type', 'fixed acidity', 'volatile acidity']]
target = 'quality'

# Standardize features
df_train[features] = scaler.fit_transform(df_train[features])
df_test[features] = scaler.transform(df_test[features])

In [4]:
# Tạo cột mới bằng cách nhân 'fixed acidity' với 'volatile acidity'
df_train['new_column'] = df_train['fixed acidity'] * df_train['volatile acidity']
df_test['new_column'] = df_test['fixed acidity'] * df_test['volatile acidity']

# Xóa cột 'fixed acidity' và 'volatile acidity'
df_train.drop(['fixed acidity', 'volatile acidity'], axis=1, inplace=True)
df_test.drop(['fixed acidity', 'volatile acidity'], axis=1, inplace=True)

# Update feature list after feature engineering
features = [col for col in df_train.columns if col not in ['quality', 'id']]

In [5]:
# Tính thời gian chạy code và bộ nhớ sử dụng
process = psutil.Process()
start_time = time.time()

In [6]:
# Gộp mỗi phần với các nhãn còn lại để huấn luyện
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # Số folds và random state
models_regression = []
for train_index, val_index in kf.split(df_train):
    quality_part = pd.concat([df_train.iloc[train_index], df_train[df_train['quality'].isin([5, 6, 7])]])

    # Gộp với các nhãn còn lại
    train_data = pd.concat([df_train[df_train['quality'] != 5], df_train[df_train['quality'] != 6], df_train[df_train['quality'] != 7], quality_part])
 
    # Define base models for regression
    base_models_regression = [("ExtraTreesRegressor", ExtraTreesRegressor(n_jobs=-1, random_state=7789)),
                              ("RandomForestRegressor", RandomForestRegressor(random_state=7789)),
                              ("GradientBoostingRegressor", GradientBoostingRegressor(random_state=7789)),
                              ("SVR", SVR())]


    # Final model using StackingRegressor
    model_regression = StackingRegressor(estimators=base_models_regression,
                                         final_estimator=ExtraTreesRegressor(n_jobs=-1, random_state=7789))


    # Train the regression model
    model_regression.fit(train_data[features], train_data[target])

    # Train the classification model

    # Add the models to the lists
    models_regression.append(model_regression)

In [7]:
# Ghi nhận thời gian kết thúc
end_time = time.time()
execution_time = end_time - start_time
# Tính toán thông tin hiệu suất
process = psutil.Process()
memory_usage = process.memory_info().rss / 1024 ** 2  # Đơn vị: Megabytes
print('Thời gian thực thi:', execution_time, 'giây')
print('Sử dụng bộ nhớ:', memory_usage, 'MB')

Thời gian thực thi: 2150.204533815384 giây
Sử dụng bộ nhớ: 981.95703125 MB


In [8]:
# Dự đoán chất lượng trên tập kiểm tra
X_test = df_test[features]

y_preds_regression = []

for model in models_regression:
    y_pred_regression = model.predict(X_test)
    y_preds_regression.append(y_pred_regression)

# Lấy trung bình của các dự đoán
ensemble_predictions = np.mean(y_preds_regression, axis=0)

In [9]:
# Tạo DataFrame mới chứa 'id' và 'quality'
output_df = pd.DataFrame({'id': df_test['id'], 'quality': ensemble_predictions})
output_df.to_csv('Essemble_8.csv',index= False)